In [4]:
#keras.preprocessing.sequence.pad_sequences(sequences, maxlen=None, dtype='int32',
#    padding='pre', truncating='pre', value=0.)

In [5]:
import numpy as np
from sklearn import preprocessing

In [6]:
def encode(labels):
    return (labels -100)/300
def decode(enc_labels):
    return (labels*300)+100

In [7]:
number_to_train = 8000
number_to_test =2000

In [10]:
#features_fixed_nu_hits_orig = np.loadtxt("features_fixed_nu_hits2.csv")
#results_orig = np.loadtxt("results2.csv")
features_fixed_nu_hits_orig = np.load("features_fixed_nu_hits2_easy.npy")
results_orig = np.load("results2_easy.npy")

In [11]:
results_enc = encode(results_orig)
features_fixed_nu_hits = features_fixed_nu_hits_orig.reshape([-1,1000,3])
print results_enc.shape, features_fixed_nu_hits.shape
print features_fixed_nu_hits

(10000,) (10000, 1000, 3)
[[[  8.04076342e-01   5.14007839e+00  -2.89010094e+02]
  [ -5.20139323e+00  -7.85420461e-01   1.75958459e+02]
  [ -1.16234310e+00  -5.13203462e+00   4.96997006e+02]
  ..., 
  [ -1.55646975e+01   1.72426391e+01  -2.38978672e+02]
  [  2.18445338e+01  -8.18394347e+00   1.88000276e+02]
  [ -1.38551496e+01  -1.88109141e+01   7.60128975e+01]]

 [[  5.22592743e+00   3.67916895e-01  -2.27004249e+02]
  [ -6.25112915e-01  -5.21428249e+00   1.30283544e+01]
  [ -3.27319620e-01  -5.29447848e+00   1.12987782e+02]
  ..., 
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]

 [[  5.17388936e+00  -4.17068877e-01  -2.01980751e+02]
  [ -5.14501858e+00   1.42360817e+00  -2.13993046e+02]
  [ -6.38472762e-01  -5.43867831e+00   1.29739377e+01]
  ..., 
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
  [  0.0000

In [ ]:
scaler = preprocessing.StandardScaler()
a = features_fixed_nu_hits

c = np.array([a[:,:,0].flatten(), a[:,:,1].flatten(), a[:,:,2].flatten()]).T  ## collect all x , y and z values of all hits
scaler.fit(c)

for i in range(a.shape[0]):
    tdata_contrib = scaler.transform(a[i,:,:])
    if i==0:
        transformed_data = [tdata_contrib]
    else:
        transformed_data = np.append(transformed_data, [tdata_contrib], axis=0)
        
    


assert features_fixed_nu_hits.shape == transformed_data.shape
features_fixed_nu_hits = transformed_data
print features_fixed_nu_hits

In [13]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model


#fix number of hits
# order in ascending value of rho


from keras.layers import Input, Dense, Dropout, Flatten 
from keras.layers import Conv3D, MaxPooling3D
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(1000,3,)) # (number of hits per event), (x,y,z)
#labels shape (,1)

x = LSTM(256, return_sequences=False)(inputs)
#x = Flatten()(inputs)



# a layer instance is callable on a tensor, and returns a tensor

x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='tanh')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='mse',   #mean squared error
              metrics=[])
model.fit(features_fixed_nu_hits[:number_to_train,:,:], results_enc[:number_to_train], epochs=2, batch_size=25)  # starts training




Epoch 1/2
8000/8000 [==============================] - 4742s - loss: 0.1130   
Epoch 2/2
8000/8000 [==============================] - 4170s - loss: 0.1136   


In [14]:
model.evaluate(features_fixed_nu_hits[number_to_train:number_to_train+number_to_test,:], results_enc[number_to_train:number_to_train+number_to_test], batch_size=25)

2000/2000 [==============================] - 61s    


0.11113754259422422

In [ ]:
predicted = model.predict(features_fixed_nu_hits[number_to_train:number_to_train+number_to_test,:])

predicted = model.predict(features_fixed_nu_hits[number_to_train:number_to_train+number_to_test,:])

In [ ]:
print predicted, results_enc[number_to_train:number_to_train+number_to_test]